In [3]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [4]:
# 라이브러리 선언
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

In [5]:
# OpenAI 객체를 생성합니다.
model = ChatOpenAI(temperature=0, model_name="gpt-4.1-mini")

In [28]:
# 원하는 데이터 구조를 정의합니다.
class Filter(BaseModel):
    SALE_PRC_GTE: int = Field(description="가격범위 최소값. 예를 들어 100만원 이상 120만원 이하이면, 1000000")
    SALE_PRC_LTE: int = Field(description="가격범위 최대값. 예를 들어 100만원 이상 120만원 이하이면, 1200000")
    BRND_NM: str = Field(description="브랜드명")
    ARTC_NM: str = Field(description="품목")
    CATEGORY_NM: str = Field(description="카테고리명")
    FEATURES: str = Field(description="주요기능")

# {'SALE_PRC': {'$gte': 1000000, '$lte': 1200000}}

In [29]:
# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Filter)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"SALE_PRC_GTE": {"description": "가격범위 최소값. 예를 들어 100만원 이상 120만원 이하이면, 1000000", "title": "Sale Prc Gte", "type": "integer"}, "SALE_PRC_LTE": {"description": "가격범위 최대값. 예를 들어 100만원 이상 120만원 이하이면, 1200000", "title": "Sale Prc Lte", "type": "integer"}, "BRND_NM": {"description": "브랜드명", "title": "Brnd Nm", "type": "string"}, "ARTC_NM": {"description": "품목", "title": "Artc Nm", "type": "string"}, "CATEGORY_NM": {"description": "카테고리명", "title": "Category Nm", "type": "string"}, "FEATURES": {"description": "주요기능", "title": "Features

In [30]:

# 프롬프트를 생성합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 가전 전문 e커머스 검색 AI 어시스턴트 입니다. 사용자의 자연어 검색어를 분석하여, 그 의도에 맞는 최적의 검색 필터 조건을 완성해야 합니다."),
        ("user", "#Format: {format_instructions}\n\n#Question: {query}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 체인을 구성합니다.
chain = prompt | model | parser

In [31]:
# 체인을 호출하여 쿼리 실행
queries = []
intents = []

query = "5만원 이상 블루투스 마우스"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")
queries.append(query)
intents.append(answer)

query = "4인 가족 냉장고 추천"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")
queries.append(query)
intents.append(answer)

query = "원룸용 세탁기"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")
queries.append(query)
intents.append(answer)

query = "게이밍용 고성능 모니터"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")
queries.append(query)
intents.append(answer)

query = "신혼부부용 가전세트"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")
queries.append(query)
intents.append(answer)

query = "로보락 로봇청소기 추천"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")
queries.append(query)
intents.append(answer)

query = "4K HDR 지원 55인치 TV"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")
queries.append(query)
intents.append(answer)

query = "20만원대 스팀 다리미"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")
queries.append(query)
intents.append(answer)

query = "인덕션 3구 전기레인지"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")
queries.append(query)
intents.append(answer)

query = "조용하고 전기요금 적게 나오는 에어컨"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")
queries.append(query)
intents.append(answer)

query = "에어컨 클리닝"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")
queries.append(query)
intents.append(answer)

5만원 이상 블루투스 마우스: {'SALE_PRC_GTE': 50000, 'SALE_PRC_LTE': 10000000, 'BRND_NM': '', 'ARTC_NM': '마우스', 'CATEGORY_NM': '컴퓨터 주변기기', 'FEATURES': '블루투스'}
4인 가족 냉장고 추천: {'SALE_PRC_GTE': 0, 'SALE_PRC_LTE': 10000000, 'BRND_NM': '', 'ARTC_NM': '냉장고', 'CATEGORY_NM': '가전', 'FEATURES': '4인 가족용'}
원룸용 세탁기: {'SALE_PRC_GTE': 0, 'SALE_PRC_LTE': 1000000, 'BRND_NM': '', 'ARTC_NM': '세탁기', 'CATEGORY_NM': '가전', 'FEATURES': '원룸용, 소형, 컴팩트'}
게이밍용 고성능 모니터: {'SALE_PRC_GTE': 300000, 'SALE_PRC_LTE': 1000000, 'BRND_NM': '', 'ARTC_NM': '모니터', 'CATEGORY_NM': '컴퓨터/노트북/주변기기', 'FEATURES': '게이밍, 고성능'}
신혼부부용 가전세트: {'SALE_PRC_GTE': 500000, 'SALE_PRC_LTE': 3000000, 'BRND_NM': '', 'ARTC_NM': '가전세트', 'CATEGORY_NM': '신혼부부용 가전', 'FEATURES': '신혼부부용, 세트구성, 필수가전'}
로보락 로봇청소기 추천: {'SALE_PRC_GTE': 0, 'SALE_PRC_LTE': 10000000, 'BRND_NM': '로보락', 'ARTC_NM': '로봇청소기', 'CATEGORY_NM': '청소기', 'FEATURES': ''}
4K HDR 지원 55인치 TV: {'SALE_PRC_GTE': 0, 'SALE_PRC_LTE': 100000000, 'BRND_NM': '', 'ARTC_NM': 'TV', 'CATEGORY_NM': 'TV', 'FEATURES': '4K HD

In [ ]:
query = "삼성"
answer = chain.invoke({"query": query})
queries.append(query)
intents.append(answer)

import re

for intent in intents):
    # print(f"{query} : {intent}")

    filter_dict = {}

    # 가격
    if "SALE_PRC" not in filter_dict:
        filter_dict["SALE_PRC"] = {}

    filter_dict["SALE_PRC"]["$gte"] = intent["SALE_PRC_GTE"]
    filter_dict["SALE_PRC"]["$lte"] = intent["SALE_PRC_LTE"]

    # 브랜드
    filter_dict["BRND_NM"] = intent["BRND_NM"]

    # 품목
    filter_dict["ARTC_NM"] = intent["ARTC_NM"]

    # 카테고리
    filter_dict["CATEGORY_NM"] = intent["CATEGORY_NM"]

    # 주요 기능 및 특징
    filter_dict["FEATURES"] = intent["FEATURES"]

    print(filter_dict)

{'SALE_PRC': {'$gte': 50000, '$lte': None}, 'BRND_NM': '', 'ARTC_NM': '마우스', 'CATEGORY_NM': '컴퓨터 주변기기', 'FEATURES': '블루투스'}
{'SALE_PRC': {'$gte': 500000, '$lte': 3000000}, 'BRND_NM': '', 'ARTC_NM': '냉장고', 'CATEGORY_NM': '주방가전', 'FEATURES': '4인 가족용, 대용량'}
{'SALE_PRC': {'$gte': 100000, '$lte': 500000}, 'BRND_NM': '', 'ARTC_NM': '세탁기', 'CATEGORY_NM': '생활가전', 'FEATURES': '원룸용, 소형'}
{'SALE_PRC': {'$gte': 500000, '$lte': 2000000}, 'BRND_NM': '', 'ARTC_NM': '모니터', 'CATEGORY_NM': '컴퓨터/노트북/주변기기', 'FEATURES': '게이밍, 고성능'}
{'SALE_PRC': {'$gte': 500000, '$lte': 3000000}, 'BRND_NM': '', 'ARTC_NM': '가전세트', 'CATEGORY_NM': '신혼부부용', 'FEATURES': '기본가전, 실용적, 경제적'}
{'SALE_PRC': {'$gte': 0, '$lte': 10000000}, 'BRND_NM': '국산', 'ARTC_NM': '로봇청소기', 'CATEGORY_NM': '청소기', 'FEATURES': ''}
{'SALE_PRC': {'$gte': 0, '$lte': 100000000}, 'BRND_NM': '', 'ARTC_NM': 'TV', 'CATEGORY_NM': '55인치', 'FEATURES': '4K HDR'}
{'SALE_PRC': {'$gte': 200000, '$lte': 299999}, 'BRND_NM': '', 'ARTC_NM': '다리미', 'CATEGORY_NM': '스팀다리미', 'F